# lecture

## 10.1 ReLU

- XSigmoid와 ReLU 비교 : sigmoid를 이용할 때 그 값은 항상 1보다 작고 층이 많아지면 이는 계속 곱해지므로 0으로 가까워지게 된다. 즉, hidden층이 점점 많아질수록 말단에서 부터 점점 경사도가 줄어들게 되어 첫단은 거의 0에 가까워지는 현상이 발생하게 된다. 따라서 이를 보완하기 위해 ReLU(rectified linear unit)라는 것을 이용하여 1보다 큰 값도 존재하도록 한다. (그래프가 s자 형태에서 일차형 직선으로 변하게 된다.) 결과적으로는 cost에서 sigmoid는 학습을 계속해도 일정하지만 ReLU에서는 0으로 떨어지게 된다.

## 10.2 초기값 설정하기(Weight)
:0으로 초기값을 줄 경우, 처음에 x에 대한 미분값과 w값을 곱해주게 되는데 이 부분에서 항상 0으로 나오기 때문에 문제가 발생한다. 즉, gradient가 사라짐

=> 0은 사용해서는 안 되고 이 때 사용하는 것이 RBM(Restricted Boltzmann Machine)이다.

- RBM(2006) : Forward와 Backward로 나누고 결과가 나온 x값을 비교해서 최대한 비슷하게 되도록 weight을 조정하는 것이다.
        - forward : x를 통해 어떤 값을 만들어 낸다.
        - bacward : forward에서 나온 값으로 역으로 새로운 x를 생성합니다.
=> 이를 구현하려면 층을 2개씩만 생각해서 시작부터 학습해서 끝까지 구한다.(pre-training), 이로 나온 결과가 FINE TUNING이다.

- xavier initialization(2010)/he initialization(2015) : RBM으로 초기화 필요없이 weight값을 너무 작지도 크지도 않게 적절히 만들고 fan_in과 fan_out값을 사용하여 weight을 설정할 수 있다.

## 10.3 dropout과 앙상블

- overfitting : 값들을 기준에 의해 나눌 때 하나하나 신경쓰게 되어 그래프가 막 휘게 되는 현상으로 새로운 test를 했을 때 정확도가 제대로 나오지 않는 현상이 발생한다. regularizaion에서는 weight의 값을 너무 크게 하지 않음으로써 이를 해결했고 Neural Network에서는 Dropout이라는 것을 사용한다.

- Dropout : neuron들 중에서 랜덤하게 몇개를 끊어서 생각하는 방식이다. 구현할 때 train 부분에서 dropout_rate을 퍼센트로 설정해주고 evaluation부분에서 dropout_rate를 1로 즉, 전체를 다루도록 한다. (랜덤 포레스트와 비슷)

## 10.4 network 모듈 쌓기

- fast forward : 여러개 층의 네트워크가 존재할 때 몇 단계의 네트워크를 건너뛰고 결과를 구하는 것이다.
- split & merge : 두 갈래로 나뉘었다가 합쳐질 수 있고 여러개의 데이터가 들어와서 하나로 합쳐지는 구조가 될 수 있다.
- Recurrent network : 직선으로만 network를 구성하는 것이 아닌 옆으로도 뻗어나가는 구조이다. 한 예로 RNN이 있다.

 # programming

## 10.5 딥러닝으로 MNIST 98%이상 만들기

- NN for MNIST

In [1]:
import tensorflow as tf
import random

from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("MNIST_data/", one_hot=True)

learning_rate = 0.001
training_epochs = 15
batch_size = 100

X = tf.placeholder(tf.float32, [None, 784])
Y = tf.placeholder(tf.float32, [None, 10])

W1 = tf.Variable(tf.random_normal([784, 256]))
b1 = tf.Variable(tf.random_normal([256]))
L1 = tf.nn.relu(tf.matmul(X, W1) + b1)

W2 = tf.Variable(tf.random_normal([256, 256]))
b2 = tf.Variable(tf.random_normal([256]))
L2 = tf.nn.relu(tf.matmul(L1, W2) + b2)

W3 = tf.Variable(tf.random_normal([256, 10]))
b3 = tf.Variable(tf.random_normal([10]))
hypothesis = tf.matmul(L2, W3) + b3

cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(
    logits=hypothesis, labels=Y))
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(cost)

# initialize
sess = tf.Session()
sess.run(tf.global_variables_initializer())

# train
for epoch in range(training_epochs):
    avg_cost = 0
    total_batch = int(mnist.train.num_examples / batch_size)

    for i in range(total_batch):
        batch_xs, batch_ys = mnist.train.next_batch(batch_size)
        feed_dict = {X: batch_xs, Y: batch_ys}
        c, _ = sess.run([cost, optimizer], feed_dict=feed_dict)
        avg_cost += c / total_batch

    print('Epoch:', '%04d' % (epoch + 1), 'cost =', '{:.9f}'.format(avg_cost))

print('Learning Finished!')

# Test model and check accuracy
correct_prediction = tf.equal(tf.argmax(hypothesis, 1), tf.argmax(Y, 1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
print('Accuracy:', sess.run(accuracy, feed_dict={
      X: mnist.test.images, Y: mnist.test.labels}))

r = random.randint(0, mnist.test.num_examples - 1)
print("Label: ", sess.run(tf.argmax(mnist.test.labels[r:r + 1], 1)))
print("Prediction: ", sess.run(
    tf.argmax(hypothesis, 1), feed_dict={X: mnist.test.images[r:r + 1]}))

Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Please write your own downloading logic.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting MNIST_data/train-images-idx3-ubyte.gz
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Instructions for updating:
Please use tf.one_hot on tensors.
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See `tf.nn.softmax_cross_entropy_with_logits_v2`.

Epoch: 0001 cost = 157.412426491
Epoch: 0002 cost = 40.343098377
Epoch: 0003 cost = 25.577865525
Epoch: 0004 c

- Xavier for MNIST

In [2]:
from tensorflow.examples.tutorials.mnist import input_data
tf.set_random_seed(777)  # reproducibility
mnist = input_data.read_data_sets("MNIST_data/", one_hot=True)

learning_rate = 0.001
training_epochs = 15
batch_size = 100

X = tf.placeholder(tf.float32, [None, 784])
Y = tf.placeholder(tf.float32, [None, 10])

W1 = tf.get_variable("W1", shape=[784, 256],
                     initializer=tf.contrib.layers.xavier_initializer())
b1 = tf.Variable(tf.random_normal([256]))
L1 = tf.nn.relu(tf.matmul(X, W1) + b1)

W2 = tf.get_variable("W2", shape=[256, 256],
                     initializer=tf.contrib.layers.xavier_initializer())
b2 = tf.Variable(tf.random_normal([256]))
L2 = tf.nn.relu(tf.matmul(L1, W2) + b2)

W3 = tf.get_variable("W3", shape=[256, 10],
                     initializer=tf.contrib.layers.xavier_initializer())
b3 = tf.Variable(tf.random_normal([10]))
hypothesis = tf.matmul(L2, W3) + b3

cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(
    logits=hypothesis, labels=Y))
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(cost)

# initialize
sess = tf.Session()
sess.run(tf.global_variables_initializer())

# train
for epoch in range(training_epochs):
    avg_cost = 0
    total_batch = int(mnist.train.num_examples / batch_size)
    for i in range(total_batch):
        batch_xs, batch_ys = mnist.train.next_batch(batch_size)
        feed_dict = {X: batch_xs, Y: batch_ys}
        c, _ = sess.run([cost, optimizer], feed_dict=feed_dict)
        avg_cost += c / total_batch
    print('Epoch:', '%04d' % (epoch + 1), 'cost =', '{:.9f}'.format(avg_cost))
print('Learning Finished!')

correct_prediction = tf.equal(tf.argmax(hypothesis, 1), tf.argmax(Y, 1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
print('Accuracy:', sess.run(accuracy, feed_dict={
      X: mnist.test.images, Y: mnist.test.labels}))

r = random.randint(0, mnist.test.num_examples - 1)
print("Label: ", sess.run(tf.argmax(mnist.test.labels[r:r + 1], 1)))
print("Prediction: ", sess.run(tf.argmax(hypothesis, 1), feed_dict={X: mnist.test.images[r:r + 1]}))

Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz
Epoch: 0001 cost = 0.314990553
Epoch: 0002 cost = 0.115526457
Epoch: 0003 cost = 0.075587565
Epoch: 0004 cost = 0.054410485
Epoch: 0005 cost = 0.040330042
Epoch: 0006 cost = 0.030417863
Epoch: 0007 cost = 0.024304557
Epoch: 0008 cost = 0.019690636
Epoch: 0009 cost = 0.017158743
Epoch: 0010 cost = 0.015146158
Epoch: 0011 cost = 0.012891546
Epoch: 0012 cost = 0.009549441
Epoch: 0013 cost = 0.011842860
Epoch: 0014 cost = 0.011448498
Epoch: 0015 cost = 0.007925451
Learning Finished!
Accuracy: 0.979
Label:  [3]
Prediction:  [3]


- Dropout for MNIST(98% 이상으로 올리기)

In [4]:
tf.set_random_seed(777)  # reproducibility

mnist = input_data.read_data_sets("MNIST_data/", one_hot=True)

learning_rate = 0.001
training_epochs = 15
batch_size = 100
total_batch = int(mnist.train.num_examples / batch_size)

X = tf.placeholder(tf.float32, [None, 784])
Y = tf.placeholder(tf.float32, [None, 10])

keep_prob = tf.placeholder(tf.float32)

w1 = tf.get_variable("w1", shape=[784, 512],
                     initializer=tf.contrib.layers.xavier_initializer())
b1 = tf.Variable(tf.random_normal([512]))
L1 = tf.nn.relu(tf.matmul(X, w1) + b1)
L1 = tf.nn.dropout(L1, keep_prob=keep_prob)

w2 = tf.get_variable("w2", shape=[512, 512],
                     initializer=tf.contrib.layers.xavier_initializer())
b2 = tf.Variable(tf.random_normal([512]))
L2 = tf.nn.relu(tf.matmul(L1, w2) + b2)
L2 = tf.nn.dropout(L2, keep_prob=keep_prob)

w3 = tf.get_variable("w3", shape=[512, 512],
                     initializer=tf.contrib.layers.xavier_initializer())
b3 = tf.Variable(tf.random_normal([512]))
L3 = tf.nn.relu(tf.matmul(L2, w3) + b3)
L3 = tf.nn.dropout(L3, keep_prob=keep_prob)

w4 = tf.get_variable("w4", shape=[512, 512],
                     initializer=tf.contrib.layers.xavier_initializer())
b4 = tf.Variable(tf.random_normal([512]))
L4 = tf.nn.relu(tf.matmul(L3, w4) + b4)
L4 = tf.nn.dropout(L4, keep_prob=keep_prob)

w5 = tf.get_variable("w5", shape=[512, 10],
                     initializer=tf.contrib.layers.xavier_initializer())
b5 = tf.Variable(tf.random_normal([10]))
hypothesis = tf.matmul(L4, w5) + b5

cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(
    logits=hypothesis, labels=Y))
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(cost)

# initialize
sess = tf.Session()
sess.run(tf.global_variables_initializer())

# train
for epoch in range(training_epochs):
    avg_cost = 0

    for i in range(total_batch):
        batch_xs, batch_ys = mnist.train.next_batch(batch_size)
        feed_dict = {X: batch_xs, Y: batch_ys, keep_prob: 0.7}
        c, _ = sess.run([cost, optimizer], feed_dict=feed_dict)
        avg_cost += c / total_batch
    print('Epoch:', '%04d' % (epoch + 1), 'cost =', '{:.9f}'.format(avg_cost))
print('Learning Finished!')

correct_prediction = tf.equal(tf.argmax(hypothesis, 1), tf.argmax(Y, 1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
print('Accuracy:', sess.run(accuracy, feed_dict={
      X: mnist.test.images, Y: mnist.test.labels, keep_prob: 1})) # 반드시 keep_prob는 1로

r = random.randint(0, mnist.test.num_examples - 1)
print("Label: ", sess.run(tf.argmax(mnist.test.labels[r:r + 1], 1)))
print("Prediction: ", sess.run(
    tf.argmax(hypothesis, 1), feed_dict={X: mnist.test.images[r:r + 1], keep_prob: 1}))

Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz
Epoch: 0001 cost = 0.470693838
Epoch: 0002 cost = 0.175100109
Epoch: 0003 cost = 0.131647750
Epoch: 0004 cost = 0.109395801
Epoch: 0005 cost = 0.091504284
Epoch: 0006 cost = 0.083856108
Epoch: 0007 cost = 0.075829945
Epoch: 0008 cost = 0.069260347
Epoch: 0009 cost = 0.063235944
Epoch: 0010 cost = 0.057398760
Epoch: 0011 cost = 0.055814132
Epoch: 0012 cost = 0.049841610
Epoch: 0013 cost = 0.048327888
Epoch: 0014 cost = 0.048472135
Epoch: 0015 cost = 0.044900795
Learning Finished!
Accuracy: 0.9822
Label:  [2]
Prediction:  [2]
